<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/modeling/optimization_elasticnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtrader

     |████████████████████████████████| 410 kB 5.2 MB/s 


In [2]:
!pip install backtesting

     |████████████████████████████████| 175 kB 6.0 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173822 sha256=399af13eca654ccde80849bc1bfbccd2b4fe2c8c9fdff9a2e07c88294a2508c2
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [3]:
import numpy as np
from google.colab import drive
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
import pandas as pd
import backtrader as bt
from backtrader.feeds import PandasData
import backtrader.indicators as btind
from sklearn.metrics import mean_squared_error

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import json
with open('/content/drive/MyDrive/[Shared]K-Project/data/initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

In [6]:
mktcap_top = ['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']

In [7]:
def datasplit(df, Y_colname, X_colname): #데이터 분리 함수
  df_train = df.loc[0 : round(len(df) * 0.75)] 
  df_test = df.loc[round(len(df) * 0.75) + 1 : ]
  Y_train = df_train[Y_colname]
  X_train = df_train[X_colname]
  Y_test = df_test[Y_colname]
  X_test = df_test[X_colname]
  return X_train, X_test, Y_train, Y_test

In [8]:
class PandasData_Signal(PandasData):

  lines = ('signal',)

  # add the parameter to the parameters inherited from the base class

  params = (('signal', 6),)

class MLSignal(bt.SignalStrategy):

    def log(self, txt, dt=None):

        ''' Logging function fot this strategy'''

        dt = dt or self.datas[0].datetime.date(0)

        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.signal = self.datas[0].signal
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))
        
    def next(self):
        #Check if we are in the market 
        if not self.position :            
            if self.signal[0] == -1:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                #Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
                
        else:           
            #Already in the market...we might sell                     
            if self.signal[0] == 1:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [ ]:
final_df = pd.DataFrame([], columns = ['full_code', 'train', 'test', 'max_iter', 'random_state', 'l1_ratio', 'alpha', 'MDD', 'CAGR', 'profit'])

for i in range(len(mktcap_top)):
# for i in range(1):
  mktcap_data = pd.read_csv('/content/drive/MyDrive/[Shared]K-Project/data/mktcap_top10/{}.csv'.format(mktcap_top[i]))
  mktcap_data['TREND'] = mktcap_data['TREND'].fillna(0)

  #최신 데이터가 뒤로 오도록 정렬
  mktcap_data = mktcap_data.sort_values(by=['TRD_DD'])
  mktcap_data.set_index('TRD_DD',drop=True,inplace=True)
  mktcap_data.reset_index('TRD_DD',drop=False,inplace=True)
  print(mktcap_data)
  Y_colname = ['TREND'] #종속변수
  X_remove = ['TRD_DD']
  X_colname = [x for x in mktcap_data.columns if x not in Y_colname + X_remove] #입력변수들

  X_train, X_test, Y_train, Y_test = datasplit(mktcap_data, Y_colname, X_colname)

  optimum_df = pd.DataFrame([], columns = ['train', 'test', 'max_iter', 'random_state', 'l1_ratio', 'alpha', 'MDD', 'CAGR', 'profit'])
  
  headers = ['train', 'test', 'max_iter', 'random_state', 'l1_ratio', 'alpha', 'MDD', 'CAGR', 'profit']
  profit_df = pd.DataFrame([], columns = headers)


  for mi in [5000000]:
      for rs in [42]:
          for l1 in [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9]:
              for al in [0.00001,0.00005,0.0001,0.0005,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]:
                  model = ElasticNet(
                      max_iter = mi,
                      random_state = rs,
                      l1_ratio = l1,
                      alpha = al,)
                  model.fit(X_train, Y_train)

                  past = model.predict(X_test)

                  Y_train_pred = model.predict(X_train)
                  Y_test_pred = model.predict(X_test)


                  Y_test_array = np.array(Y_test).reshape(-1,1)
                  real = list(Y_test_array.flatten())
                  Y_test_pred_array = np.array(Y_test_pred).reshape(-1,1)
                  pred = list(Y_test_pred_array.flatten())
                  # print('1--------------------------------------------------')
                  # print(Y_test_pred)
                  # print('\n')

                  score_table = pd.DataFrame({'real' : real, 'pred' : Y_test_pred})


                  a = list(X_train.shape)
                  b = list(X_test.shape)
                  test_data = mktcap_data.iloc[a[0]:]

                  test_score = test_data.copy()
                  test_score['pred'] = Y_test_pred
                  test_score = test_score[['TRD_DD', 'pred']]
                  test_score = test_score.reset_index()
                  test_score = test_score.drop(['index'], axis = 1)
                  # print('2--------------------------------------------------')
                  # print(test_score)

                  for j in range(len(test_score)):
                    if test_score['pred'][j] < -0.1:
                      test_score['pred'][j] = -1
                    elif test_score['pred'][j] > 0.1:
                      test_score['pred'][j] = 1
                    else:
                      test_score['pred'][j] = 0
                  print('\n', test_score.value_counts(['pred']), '--------------------------------')

                  df = pd.read_json(json_data[mktcap_top[i]], orient ='index') # 첫번째 키값으로 데이터프레임 변환
                  json_df = df.transpose()
                  json_df = json_df[['TRD_DD', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'TDD_CLSPRC', 'ACC_TRDVOL']]
                  json_df = json_df.sort_values(by=['TRD_DD'])
                  json_df.reset_index(drop=True,inplace=True)

                  df_end = json_df.index[json_df['TRD_DD'] == '2021/12/30'][0]
                  raw = json_df.iloc[a[0]:df_end+1]
                  raw.reset_index(drop = True, inplace = True)
                  raw['TRD_DD'] = pd.to_datetime(raw['TRD_DD'])
                  
                  final = pd.merge(raw, test_score, left_index = True, right_index = True).drop(columns='TRD_DD_y')
                  final = final.rename(columns={'TRD_DD_x': 'TRD_DD'})
                  final['TDD_OPNPRC'] = final['TDD_OPNPRC'].str.replace(',','').astype('float')
                  final['TDD_HGPRC'] = final['TDD_HGPRC'].str.replace(',','').astype('float')
                  final['TDD_LWPRC'] = final['TDD_LWPRC'].str.replace(',','').astype('float')
                  final['TDD_CLSPRC'] = final['TDD_CLSPRC'].str.replace(',','').astype('float')
                  final['ACC_TRDVOL'] = final['ACC_TRDVOL'].str.replace(',','').astype('float')
                  final = final.astype({'pred':'int'})
                  # print(final.isnull().sum())
                  # print('3--------------------------------------------------')
                  

                  # 세레브로 (벡트레이더 엔진) 설정

                  # 세레브로 가져오기
                  cerebro = bt.Cerebro()

                  # 데이터 가져오기
                  data = PandasData_Signal(dataname = final,
                                  datetime = 0,
                                  open = 1,
                                  high = 2,
                                  low = 3,
                                  close = 4,
                                  volume = 5,
                                  openinterest = -1,
                                  signal = 6,
                                  )

                  # 데이터 추가하기
                  cerebro.adddata(data)

                  # 전략 추가하기
                  cerebro.addstrategy(MLSignal)

                  # 브로커 설정
                  cerebro.broker.setcash(1000000)

                  # 매매 단위 설정
                  cerebro.addsizer(bt.sizers.SizerFix, stake = 30) #한번에 30주 설정.

                  # 3. 세레브로 실행하기

                  # 초기 투자금 가져오기
                  init_cash = cerebro.broker.getvalue()

                  # 평가지표 추가
                  cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
                  cerebro.addanalyzer(bt.analyzers.Returns)
                  cerebro.addanalyzer(bt.analyzers.DrawDown)

                  # 세레브로 실행하기
                  results = cerebro.run()

                  # 최종 금액 가져오기
                  final_cash = cerebro.broker.getvalue()

                  analyzers = results[0].analyzers
                  mdd = analyzers.drawdown.get_analysis()['max']['drawdown']
                  cagr = analyzers.returns.get_analysis()['rnorm100']
                  # sharp = analyzers.sharperatio.get_analysis()['sharperatio']
                  print(f"mdd : {mdd:.2f}, cagr: {cagr:.2f}")

                  # print('init_cash: ', init_cash)
                  # print('final_cash: ', final_cash)

                  profit = float(final_cash - init_cash)/float(init_cash) * 100.
                  print("최종금액 : ", final_cash, "원")
                  print("수익률 : ", profit, "%")
                  print("\n")

                  profit_list = [a[0], b[0], mi, rs, l1, al, mdd, cagr, profit]
                  profit_df = profit_df.append(pd.Series(profit_list, index = headers), ignore_index = True)
                  # print('profit_df')
                  # print(profit_df)

  max_profit = profit_df[profit_df['profit'] == profit_df['profit'].max()].reset_index(drop = True)


  num_max_profit = pd.concat([max_profit], axis = 1)
  optimum_df = pd.concat([optimum_df, num_max_profit], axis = 0).reset_index(drop = True)
  # print(optimum_df)

  op_list = optimum_df.iloc[0].tolist()
  op_list.insert(0, mktcap_top[i])
  print(op_list)

  final_df = final_df.append(pd.Series(op_list, index=final_df.columns), ignore_index=True)

print(final_df)

final_df.to_csv('/content/drive/MyDrive/k-project/elasticnet0203.csv', encoding = 'UTF-8', index = False)


출력 삭제

In [16]:
final_df # 최종결과물

,full_code,train,test,max_iter,random_state,l1_ratio,alpha,MDD,CAGR,profit
0,KR7060310000,3654.0,1217.0,5000000.0,42.0,0.30,0.00001,3.716537,3.584543,18.540
1,KR7095570008,1175.0,390.0,5000000.0,42.0,0.30,0.00001,4.397305,3.646859,5.700
2,KR7006840003,4149.0,1381.0,5000000.0,42.0,0.30,0.00001,12.004850,18.030467,148.050
3,KR7054620000,3723.0,1240.0,5000000.0,42.0,0.70,0.00400,5.539910,18.044551,126.210
4,KR7265520007,873.0,290.0,5000000.0,42.0,0.65,0.01000,7.474128,48.726114,57.900
5,KR7211270004,1077.0,358.0,5000000.0,42.0,0.30,0.00001,4.682406,38.404337,58.680
6,KR7027410000,1408.0,468.0,5000000.0,42.0,0.30,0.00001,3.713316,9.403814,18.165
7,KR7282330000,751.0,249.0,5000000.0,42.0,0.30,0.00001,0.000000,0.000000,0.000
8,KR7138930003,1991.0,662.0,5000000.0,42.0,0.30,0.00001,4.119500,4.264549,11.595
9,KR7001460005,5067.0,1688.0,5000000.0,42.0,0.30,0.00001,0.000000,0.000000,0.000
